In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git


import sklearn
import sklearn.feature_extraction.text

#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

In [2]:
df = pandas.read_csv('t\df_80.csv', index_col=0)

df70 = pandas.read_csv('t\df_70.csv', index_col=0)
df80 = pandas.read_csv('t\df_80.csv', index_col=0)
df90 = pandas.read_csv('t\df_90.csv', index_col=0)
df00 = pandas.read_csv('t\df_00.csv', index_col=0)
df10 = pandas.read_csv('t\df_10.csv', index_col=0)

dd = pandas.concat([df70, df80, df90, df00, df10])

In [3]:
#Apply our functions
df['tokenized_text'] = df['AB'].apply(lambda x: lucem_illud_2020.word_tokenize(x))
df['normalized_tokens'] = df['tokenized_text'].apply(lambda x: lucem_illud_2020.normalizeTokens(x))

In [4]:
ngTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, 
                                                                 max_features=3000, min_df=3, 
                                                                 stop_words='english', 
                                                                 norm='l2')
groupsTFVects = ngTFVectorizer.fit_transform(df['AB'])

In [5]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

df['reduced_tokens'] = df['normalized_tokens'].apply(lambda x: dropMissing(x, ngTFVectorizer.vocabulary_.keys()))

In [6]:
dictionary = gensim.corpora.Dictionary(df['reduced_tokens'])

In [7]:
## create a list of tuples containing each token and its count 
## use the first half and save the remainder for testing 
corpus = [dictionary.doc2bow(text) for text in df['reduced_tokens']]

In [8]:
# serialize the corpus as a file and load it 
gensim.corpora.MmCorpus.serialize('abstract.mm', corpus)
abmm = gensim.corpora.MmCorpus('abstract.mm')

In [9]:
# correctly formatted corpus that can be used for topic modeling and induction 
senlda = gensim.models.ldamodel.LdaModel(corpus=abmm, id2word=dictionary, num_topics=20, alpha='auto', eta='auto')

In [10]:
ldaDF = pandas.DataFrame({
        'titles' : df['TI'],
        'topics' : [senlda[dictionary.doc2bow(l)] for l in df['reduced_tokens']]
    })

In [11]:
topicsProbDict = {i : [0] * len(ldaDF) for i in range(senlda.num_topics)}

for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob

for topicNum in range(senlda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]

In [12]:
K = senlda.num_topics  # N documents, K topics
topic_labels = ['Topic #{}'.format(k) for k in range(K)]

topicsDict = {}
for topicNum in range(senlda.num_topics):
    topicWords = [w for w, p in senlda.show_topic(topicNum)]
    topicsDict['Topic_{}'.format(topicNum)] = topicWords

wordRanksDF = pandas.DataFrame(topicsDict)
wordRanksDF

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Topic_10,Topic_11,Topic_12,Topic_13,Topic_14,Topic_15,Topic_16,Topic_17,Topic_18,Topic_19
0,patient,woman,health,health,determinant,study,family,woman,age,salt,factor,behavior,determinant,health,factor,age,patient,health,determinant,child
1,family,health,determinant,determinant,health,male,study,determinant,determinant,patient,determinant,work,support,determinant,family,dental,level,patient,factor,determinant
2,study,determinant,study,study,model,female,child,client,study,determinant,model,health,study,male,aggression,determinant,determinant,study,child,study
3,factor,age,group,self,contraceptive,determinant,factor,variable,child,low,age,determinant,mother,factor,determinant,research,school,determinant,level,patient
4,behavior,rate,care,level,care,person,determinant,behavior,population,development,level,factor,health,study,health,health,age,effect,health,age
5,size,family,change,patient,use,factor,age,analysis,health,infant,variable,age,child,level,effect,behavior,mortality,behavior,individual,high
6,important,child,patient,result,change,support,relate,sex,result,intake,significant,group,model,behavior,individual,group,self,factor,effect,health
7,predictor,use,analysis,factor,factor,infant,high,report,year,study,child,population,female,female,study,study,abortion,support,model,effect
8,relate,group,class,psychological,child,child,mother,age,status,change,suggest,life,adolescent,result,group,level,finding,high,support,rate
9,health,variable,factor,control,sexual,effect,level,measure,patient,suggest,study,level,male,care,present,child,analysis,present,high,behavior


In [13]:
import pyLDAvis.gensim
lda_model = senlda
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
1     -0.019711  0.012191       1        1  8.772115
2      0.017716 -0.018248       2        1  7.154488
8      0.015019  0.015458       3        1  6.796586
6     -0.030800 -0.028340       4        1  6.574690
12    -0.014271 -0.010500       5        1  6.331126
13    -0.021355  0.010900       6        1  5.520298
7      0.028122  0.068018       7        1  5.248167
10    -0.050340 -0.021696       8        1  5.069602
19     0.007051 -0.001774       9        1  5.038356
18    -0.039313 -0.039728      10        1  4.993696
17     0.039051 -0.062655      11        1  4.746627
5     -0.003241  0.070493      12        1  4.541792
9      0.116181 -0.013798      13        1  4.472882
0      0.025191 -0.016099      14        1  4.449900
15    -0.023416  0.045851      15        1  4.384439
11    -0.004184  0.075018      16        1  4.373270
14    -0.055598  0.010595      17        1  3.792123
4     -0.062857 -0.049278      18        1  2.902699
3      0.050235 -0.009242      19        1  2.646422
16     0.026521 -0.037166      20        1  2.190723, topic_info=     Category        Freq      Term       Total  loglift  logprob
25    Default  800.000000    health  800.000000  30.0000  30.0000
83    Default  372.000000   patient  372.000000  29.0000  29.0000
1886  Default   51.000000      salt   51.000000  28.0000  28.0000
414   Default  321.000000     woman  321.000000  27.0000  27.0000
1562  Default   91.000000    dental   91.000000  26.0000  26.0000
...       ...         ...       ...         ...      ...      ...
15    Topic20    6.680870  economic  221.656525   0.3191  -5.4278
48    Topic20    7.120736     study  656.205383  -0.7025  -5.3641
51    Topic20    6.544131   support  290.842377   0.0267  -5.4485
99    Topic20    6.466466     child  525.830811  -0.5774  -5.4605
402   Topic20    6.326526  physical  179.070129   0.4779  -5.4823

[1401 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1966      3  0.329329  1960s
1937     13  0.484425   1970
1798     10  0.295610    1st
1798     14  0.147805    1st
1798     16  0.147805    1st
...     ...       ...    ...
1476      7  0.038183  youth
1476     11  0.114549  youth
1476     12  0.076366  youth
1476     15  0.038183  youth
1476     20  0.267282  youth

[7391 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 9, 7, 13, 14, 8, 11, 20, 19, 18, 6, 10, 1, 16, 12, 15, 5, 4, 17])